In [51]:
import poplib
#解析邮件
from email.parser import Parser
from email.header import decode_header
from email.utils import parseaddr

#解析消息头中的字符串
#没有这个函数，print出来的会使乱码的头部信息。如'=?gb18030?B?yrXWpL3hufsueGxz?='这种
#通过decode，将其变为中文
def decode_str(s):
    value, charset = decode_header(s)[0]
    print(value,charset,"*"*20)
    if charset:
        value = value.decode(charset)
    return value

#解码邮件信息分为两个步骤，第一个是取出头部信息
#首先取头部信息
#主要取出['From','To','Subject']
'''
From: "=?gb18030?B?anVzdHpjYw==?=" <justonezcc@sina.com>
To: "=?gb18030?B?ztLX1Ly6tcTTys/k?=" <392361639@qq.com>
Subject: =?gb18030?B?dGV4dMTjusM=?=
'''
#如上述样式，均需要解码
def get_header(msg):
    for header in ['From', 'To', 'Subject']:
        value = msg.get(header, '')
        if value:
            #文章的标题有专门的处理方法
            if header == 'Subject':
                value = decode_str(value)
            elif header in ['From','To']:
            #地址也有专门的处理方法
                hdr, addr = parseaddr(value)
                name = decode_str(addr)
                #value = name + ' < ' + addr + ' > '
                value=name
        print(header + ':' + value)
#头部信息已取出


#获取邮件的字符编码，首先在message中寻找编码，如果没有，就在header的Content-Type中寻找
def guess_charset(msg):
    charset = msg.get_charset()
    if charset is None:
        content_type = msg.get('Content-Type', '').lower()
        pos = content_type.find('charset=')
        if pos >= 0:
            charset = content_type[pos+8:].strip()
    return charset


#邮件正文部分
#取附件
#邮件的正文部分在生成器中，msg.walk()
#如果存在附件，则可以通过.get_filename()的方式获取文件名称

def get_file(msg):
    for part in msg.walk():
        filename=part.get_filename()
        if filename!=None:#如果存在附件
            filename = decode_str(filename)#获取的文件是乱码名称，通过一开始定义的函数解码
            print("文件名 %s" %filename)
            
            data = part.get_payload(decode = True)#取出文件正文内容
            #此处可以自己定义文件保存位置
            path=filename
            f = open(path, 'wb')
            f.write(data)
            f.close()
            print(filename,'download')

def get_content(msg):
    for part in msg.walk():
        content_type = part.get_content_type()
        charset = guess_charset(part)
        #如果有附件，则直接跳过
        if part.get_filename()!=None:
            continue
        email_content_type = ''
        content = ''
        if content_type == 'text/plain':
            email_content_type = 'text'
        elif content_type == 'text/html':
            print('html 格式 跳过')
            continue #不要html格式的邮件
            email_content_type = 'html'
        if charset:
            try:
                content = part.get_payload(decode=True).decode(charset)
            except AttributeError:
                print('type error')
            except LookupError:
                print("unknown encoding: utf-8")
        if email_content_type =='':
            continue
            #如果内容为空，也跳过
        print(email_content_type + ' -----  ' + content)

#get_file(msg)
if __name__ == '__main__':
    print('start')
    mail_host="pop.qq.com"  #设置服务器
    email='wind2zero@qq.com'
    password='vclmmuvcnwntdjef'
    

    server=poplib.POP3_SSL(mail_host)

    server.user(email)
    server.pass_(password)
    #登录的过程

    resp, mails, octets = server.list()
    print(resp)
    print(mails)
    print(octets )
    print('-'*30)
    index = len(mails)#邮件的总数
    #此处的循环是取最近的几封邮件
    print(index)


    for i in range(1,index+1):
        resp, lines, octets = server.retr(i)#取邮件
        print(resp)
        print(lines)
        print(octets)
        msg_content = b'\r\n'.join(lines).decode('utf-8','ignore')
        msg = Parser().parsestr(msg_content)
        #server.dele(index) 删除邮件
        get_header(msg)
        get_file(msg)
        get_content(msg)
        print('-'*30)
    server.quit()

start
b'+OK'
[b'1 8305', b'2 8308', b'3 8518']
24
------------------------------
3
b'+OK'
[b'Date: Sat, 2 Nov 2019 16:41:18 +0800', b'X-QQ-mid: esmtp7t1572684076tzpuyvle2', b'Received: from [192.168.0.100] (unknown [175.10.95.63])', b'\tby esmtp4.qq.com (ESMTP) with SMTP id 0', b'\tfor <2437253053@qq.com>; Sat, 02 Nov 2019 16:41:16 +0800 (CST)', b'X-QQ-SSF: A001000000000010G3120000002000J', b'X-QQ-FEAT: DvNdkFXOZE+xrKCpNhH6civIzs0DRP9Dq8D6fRROQnsZyOs/UPqYKlP9LuFy6', b'\tba5T4CpBzLi7JQwXnL67wV+jsoRmkhUptPoJQLtxS1Z9yz2EeVLkcEpg+4di4EnKIG0zieH', b'\tZrIXwXw1oFAM7z8fuHni71R9S5EdvsHpt1xUhQh/NciLFPPjYPSZVGpMKesBzHIBcb2cyRg', b'\tuIRD01Fr30Bl3rX2+u4CuJ0FTYeweZCPwhSm8ntamawRWFI2L2c8LA37JJ2j0pU8=', b'X-QQ-CSender: 2437253053@qq.com', b'Sender: 2437253053@qq.com', b'Content-Type: multipart/mixed; boundary="===============2997002501849855988=="', b'MIME-Version: 1.0', b'From: =?utf-8?b?WkVST+aVmeeoiw==?=', b'To: =?utf-8?b?aml1Z3VhaeS9oOWlvQ==?=', b'Subject: =?utf-8?b?UHl0aG9uIFNNVFAg6YKu5Lu25rWL6

b'+OK'
[b'Date: Sat, 2 Nov 2019 16:54:13 +0800', b'X-QQ-mid: esmtp3t1572684852tceyu89lx', b'Received: from [192.168.0.100] (unknown [175.10.95.63])', b'\tby esmtp4.qq.com (ESMTP) with SMTP id 0', b'\tfor <2437253053@qq.com>; Sat, 02 Nov 2019 16:54:11 +0800 (CST)', b'X-QQ-SSF: A001000000000010G3120000002000J', b'X-QQ-FEAT: x/3tCoGad+b3J71CeACfdeeMfDiB2uQQib2fBf14HYPlsViz618+DsOhHPqhL', b'\tgncOt1G+UnXPaUAbFyKWa1s+MWI3TjYRcXx27h2IT6n9iyEFjb406t1ydWH2uGHiYFfm6aO', b'\txm3mzRc8CTZIs0GTi6OkRFCpNdr/eK902UHJO6Dvjok5rIaVYLqjWwuTcuLqAR9zVvLGj22', b'\tWT/3LNkPeASk90ZErVpX/j2BrvtfLT1v9oSVMkPOVAs9g/gb5b3Pvlk+O7R7Q9ojmZZcXXW', b'\tJWVYmGs25ZIf7e', b'X-QQ-CSender: 2437253053@qq.com', b'Sender: 2437253053@qq.com', b'Content-Type: multipart/mixed; boundary="===============8928907342863906369=="', b'MIME-Version: 1.0', b'From: =?utf-8?b?WkVST+aVmeeoiw==?=', b'To: =?utf-8?b?aml1Z3VhaeS9oOWlvQ==?=', b'Subject: =?utf-8?b?UHl0aG9uIFNNVFAg6YKu5Lu25rWL6K+V?=', b'', b'--===============8928907342863906369==', 

In [60]:
msg.items()

[('Date', 'Sat, 2 Nov 2019 16:54:13 +0800'),
 ('X-QQ-mid', 'esmtp3t1572684852tceyu89lx'),
 ('Received',
  'from [192.168.0.100] (unknown [175.10.95.63])\r\n\tby esmtp4.qq.com (ESMTP) with SMTP id 0\r\n\tfor <2437253053@qq.com>; Sat, 02 Nov 2019 16:54:11 +0800 (CST)'),
 ('X-QQ-SSF', 'A001000000000010G3120000002000J'),
 ('X-QQ-FEAT',
  'x/3tCoGad+b3J71CeACfdeeMfDiB2uQQib2fBf14HYPlsViz618+DsOhHPqhL\r\n\tgncOt1G+UnXPaUAbFyKWa1s+MWI3TjYRcXx27h2IT6n9iyEFjb406t1ydWH2uGHiYFfm6aO\r\n\txm3mzRc8CTZIs0GTi6OkRFCpNdr/eK902UHJO6Dvjok5rIaVYLqjWwuTcuLqAR9zVvLGj22\r\n\tWT/3LNkPeASk90ZErVpX/j2BrvtfLT1v9oSVMkPOVAs9g/gb5b3Pvlk+O7R7Q9ojmZZcXXW\r\n\tJWVYmGs25ZIf7e'),
 ('X-QQ-CSender', '2437253053@qq.com'),
 ('Sender', '2437253053@qq.com'),
 ('Content-Type',
  'multipart/mixed; boundary="===============8928907342863906369=="'),
 ('MIME-Version', '1.0'),
 ('From', '=?utf-8?b?WkVST+aVmeeoiw==?='),
 ('To', '=?utf-8?b?aml1Z3VhaeS9oOWlvQ==?='),
 ('Subject', '=?utf-8?b?UHl0aG9uIFNNVFAg6YKu5Lu25rWL6K+V?=')]

In [61]:
msg.get('sender')

'2437253053@qq.com'

In [62]:
msg.get('received')

'from [192.168.0.100] (unknown [175.10.95.63])\r\n\tby esmtp4.qq.com (ESMTP) with SMTP id 0\r\n\tfor <2437253053@qq.com>; Sat, 02 Nov 2019 16:54:11 +0800 (CST)'

In [63]:
msg.get('from')

'=?utf-8?b?WkVST+aVmeeoiw==?='

In [58]:
dir(msg)

['__bytes__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_charset',
 '_default_type',
 '_get_params_preserve',
 '_headers',
 '_payload',
 '_unixfrom',
 'add_header',
 'as_bytes',
 'as_string',
 'attach',
 'defects',
 'del_param',
 'epilogue',
 'get',
 'get_all',
 'get_boundary',
 'get_charset',
 'get_charsets',
 'get_content_charset',
 'get_content_disposition',
 'get_content_maintype',
 'get_content_subtype',
 'get_content_type',
 'get_default_type',
 'get_filename',
 'get_param',
 'get_params',
 'get_payload',
 'get_unixfrom',
 'is_multipart',
 'items',
 'keys',
 'policy',

In [56]:
lines

[b'Date: Sat, 2 Nov 2019 16:54:13 +0800',
 b'X-QQ-mid: esmtp3t1572684852tceyu89lx',
 b'Received: from [192.168.0.100] (unknown [175.10.95.63])',
 b'\tby esmtp4.qq.com (ESMTP) with SMTP id 0',
 b'\tfor <2437253053@qq.com>; Sat, 02 Nov 2019 16:54:11 +0800 (CST)',
 b'X-QQ-SSF: A001000000000010G3120000002000J',
 b'X-QQ-FEAT: x/3tCoGad+b3J71CeACfdeeMfDiB2uQQib2fBf14HYPlsViz618+DsOhHPqhL',
 b'\tgncOt1G+UnXPaUAbFyKWa1s+MWI3TjYRcXx27h2IT6n9iyEFjb406t1ydWH2uGHiYFfm6aO',
 b'\txm3mzRc8CTZIs0GTi6OkRFCpNdr/eK902UHJO6Dvjok5rIaVYLqjWwuTcuLqAR9zVvLGj22',
 b'\tWT/3LNkPeASk90ZErVpX/j2BrvtfLT1v9oSVMkPOVAs9g/gb5b3Pvlk+O7R7Q9ojmZZcXXW',
 b'\tJWVYmGs25ZIf7e',
 b'X-QQ-CSender: 2437253053@qq.com',
 b'Sender: 2437253053@qq.com',
 b'Content-Type: multipart/mixed; boundary="===============8928907342863906369=="',
 b'MIME-Version: 1.0',
 b'From: =?utf-8?b?WkVST+aVmeeoiw==?=',
 b'To: =?utf-8?b?aml1Z3VhaeS9oOWlvQ==?=',
 b'Subject: =?utf-8?b?UHl0aG9uIFNNVFAg6YKu5Lu25rWL6K+V?=',
 b'',
 b'--===============892890734286

In [46]:
l  = [b'Date: Sat, 2 Nov 2019 16:41:18 +0800', b'X-QQ-mid: esmtp7t1572684076tzpuyvle2', b'Received: from [192.168.0.100] (unknown [175.10.95.63])', b'\tby esmtp4.qq.com (ESMTP) with SMTP id 0', b'\tfor <2437253053@qq.com>; Sat, 02 Nov 2019 16:41:16 +0800 (CST)', b'X-QQ-SSF: A001000000000010G3120000002000J', b'X-QQ-FEAT: DvNdkFXOZE+xrKCpNhH6civIzs0DRP9Dq8D6fRROQnsZyOs/UPqYKlP9LuFy6', b'\tba5T4CpBzLi7JQwXnL67wV+jsoRmkhUptPoJQLtxS1Z9yz2EeVLkcEpg+4di4EnKIG0zieH', b'\tZrIXwXw1oFAM7z8fuHni71R9S5EdvsHpt1xUhQh/NciLFPPjYPSZVGpMKesBzHIBcb2cyRg', b'\tuIRD01Fr30Bl3rX2+u4CuJ0FTYeweZCPwhSm8ntamawRWFI2L2c8LA37JJ2j0pU8=', b'X-QQ-CSender: 2437253053@qq.com', b'Sender: 2437253053@qq.com', b'Content-Type: multipart/mixed; boundary="===============2997002501849855988=="', b'MIME-Version: 1.0', b'From: =?utf-8?b?WkVST+aVmeeoiw==?=', b'To: =?utf-8?b?aml1Z3VhaeS9oOWlvQ==?=', b'Subject: =?utf-8?b?UHl0aG9uIFNNVFAg6YKu5Lu25rWL6K+V?=', b'', b'--===============2997002501849855988==', b'Content-Type: application/octet-stream', b'MIME-Version: 1.0', b'Content-Transfer-Encoding: base64', b'Content-Disposition: attachment; filename="Apple.xlsx"', b'', b'UEsDBBQAAAAIAMSIYU8HQU1igQAAALEAAAAQAAAAZG9jUHJvcHMvYXBwLnhtbE2OPQsCMRBE/8px', b'vbdBwUJiQNBSsLIPexsvkGRDskJ+vjnBj24ebxhG3wpnKuKpDi2GVI/jIpIPABUXirZOXaduHJdo', b'pWN5ADvnkc6Mz0hJYKvUHqgJpZnmTf4Ojkafcg4erXhO5uqxcGUnw6UhBQ3/cm3eqdQ17yb1lh/W', b'8DtpXlBLAwQUAAAACADEiGFPzv2fo+oAAADLAQAAEQAAAGRvY1Byb3BzL2NvcmUueG1spZHBSgMx', b'EIZfpeS+O0kLLYY0F8WTgmBB8RaSaRvc7IZkZLdvb3Ztt4rePGb+b76ZIcpGabuET6mLmMhjXgyh', b'abO0ccuORFECZHvEYHJdiLaE+y4FQ+WZDhCNfTcHhCXnawhIxhkyMAqrOBvZWensrIwfqZkEzgI2', b'GLClDKIWcGUJU8h/NkzJTA7Zz1Tf93W/mriykYDXx4fnafnKt5lMa5Fp5ay0CQ11SY8XxdPQKPhW', b'VOfZXwV0izJB0inill2Sl9Xt3e6e6SUXN5UQFRc7sZF8LfnmbXT96L8KQ+f83v/DeBFoBb/+TX8C', b'UEsDBBQAAAAIAMSIYU+ZXJwjEAYAAJwnAAATAAAAeGwvdGhlbWUvdGhlbWUxLnhtbO1aW3PaOBR+', b'76/QeGf2bQvGNoG2tBNzaXbbtJmE7U4fhRFYjWx5ZJGEf79HNhDLlg3tkk26mzwELOn7zkVH5+g4', b'efPuLmLohoiU8nhg2S/b1ru3L97gVzIkEUEwGaev8MAKpUxetVppAMM4fckTEsPcgosIS3gUy9Zc', b'4FsaLyPW6rTb3VaEaWyhGEdkYH1eLGhA0FRRWm9fILTlHzP4FctUjWWjARNXQSa5iLTy+WzF/Nre', b'PmXP6TodMoFuMBtYIH/Ob6fkTlqI4VTCxMBqZz9Wa8fR0kiAgsl9lAW6Sfaj0xUIMg07Op1YznZ8', b'9sTtn4zK2nQ0bRrg4/F4OLbL0otwHATgUbuewp30bL+kQQm0o2nQZNj22q6RpqqNU0/T933f65to', b'nAqNW0/Ta3fd046Jxq3QeA2+8U+Hw66JxqvQdOtpJif9rmuk6RZoQkbj63oSFbXlQNMgAFhwdtbM', b'0gOWXin6dZQa2R273UFc8FjuOYkR/sbFBNZp0hmWNEZynZAFDgA3xNFMUHyvQbaK4MKS0lyQ1s8p', b'tVAaCJrIgfVHgiHF3K/99Ze7yaQzep19Os5rlH9pqwGn7bubz5P8c+jkn6eT101CznC8LAnx+yNb', b'YYcnbjsTcjocZ0J8z/b2kaUlMs/v+QrrTjxnH1aWsF3Pz+SejHIju932WH32T0duI9epwLMi15RG', b'JEWfyC265BE4tUkNMhM/CJ2GmGpQHAKkCTGWoYb4tMasEeATfbe+CMjfjYj3q2+aPVehWEnahPgQ', b'RhrinHPmc9Fs+welRtH2Vbzco5dYFQGXGN80qjUsxdZ4lcDxrZw8HRMSzZQLBkGGlyQmEqk5fk1I', b'E/4rpdr+nNNA8JQvJPpKkY9psyOndCbN6DMawUavG3WHaNI8ev4F+Zw1ChyRGx0CZxuzRiGEabvw', b'Hq8kjpqtwhErQj5iGTYacrUWgbZxqYRgWhLG0XhO0rQR/FmsNZM+YMjszZF1ztaRDhGSXjdCPmLO', b'i5ARvx6GOEqa7aJxWAT9nl7DScHogstm/bh+htUzbCyO90fUF0rkDyanP+kyNAejmlkJvYRWap+q', b'hzQ+qB4yCgXxuR4+5Xp4CjeWxrxQroJ7Af/R2jfCq/iCwDl/Ln3Ppe+59D2h0rc3I31nwdOLW95G', b'blvE+64x2tc0LihjV3LNyMdUr5Mp2DmfwOz9aD6e8e362SSEr5pZLSMWkEuBs0EkuPyLyvAqxAno', b'ZFslCctU02U3ihKeQhtu6VP1SpXX5a+5KLg8W+Tpr6F0PizP+Txf57TNCzNDt3JL6raUvrUmOEr0', b'scxwTh7LDDtnPJIdtnegHTX79l125COlMFOXQ7gaQr4Dbbqd3Do4npiRuQrTUpBvw/npxXga4jnZ', b'BLl9mFdt59jR0fvnwVGwo+88lh3HiPKiIe6hhpjPw0OHeXtfmGeVxlA0FG1srCQsRrdguNfxLBTg', b'ZGAtoAeDr1EC8lJVYDFbxgMrkKJ8TIxF6HDnl1xf49GS49umZbVuryl3GW0iUjnCaZgTZ6vK3mWx', b'wVUdz1Vb8rC+aj20FU7P/lmtyJ8MEU4WCxJIY5QXpkqi8xlTvucrScRVOL9FM7YSlxi84+bHcU5T', b'uBJ2tg8CMrm7Oal6ZTFnpvLfLQwJLFuIWRLiTV3t1eebnK56Inb6l3fBYPL9cMlHD+U751/0XUOu', b'fvbd4/pukztITJx5xREBdEUCI5UcBhYXMuRQ7pKQBhMBzZTJRPACgmSmHICY+gu98gy5KRXOrT45', b'f0Usg4ZOXtIlEhSKsAwFIRdy4+/vk2p3jNf6LIFthFQyZNUXykOJwT0zckPYVCXzrtomC4Xb4lTN', b'uxq+JmBLw3punS0n/9te1D20Fz1G86OZ4B6zh3OberjCRaz/WNYe+TLfOXDbOt4DXuYTLEOkfsF9', b'ioqAEativrqvT/klnDu0e/GBIJv81tuk9t3gDHzUq1qlZCsRP0sHfB+SBmOMW/Q0X48UYq2msa3G', b'2jEMeYBY8wyhZjjfh0WaGjPVi6w5jQpvQdVA5T/b1A1o9g00HJEFXjGZtjaj5E4KPNz+7w2wwsSO', b'4e2LvwFQSwMEFAAAAAgAxIhhT+ZtrvGsAwAAuRAAABgAAAB4bC93b3Jrc2hlZXRzL3NoZWV0MS54', b'bWzNmElv2zgUx8/zLQQFyLHiLjJegGaxNIcCQYt25qrYdCxUiysx9QzQD99HKpFjg4RnfEjrgynp', b'if+3/ChS1HTXdl/7jdYm+qeumn4Wb4zZXiVJv9zouujftVvdgGXddnVh4LR7TPptp4uV61RXCUFI', b'JHVRNvF86q7dd/Np+2SqstH3XdQ/1XXR/Xutq3Y3i3H8cuFj+bgx7kIyn26LR/1Jm89b6ACnyaiz', b'Kmvd9GXbRJ1ez+L3+CqXroe740upd/2r48gm89C2X+3Jn6tZjGxMutJLYyUKaL7rG11VVgki+fYs', b'Gu+d2p6vj1/kFy5/CO+h6PVNW/1VrsxmFkMwK70unirzsd3l+jknvg/xtjDFfNq1u6izyc6nS3vg', b'vLvk4faysaX6ZDqwluDPzC8vKE+ZmFxeEEwpdS1TfDJNDMRm70mWz0rXJ5WExP9J6eaUEsGSIqcg', b'VOpajlOf0u0pJaxUql7HpJQituWYUZ/i3f9WFBhZRSqplD7FxWkCREinJCTzKWQn68UVdvVCUHPX', b'Ym5bihDituUSeUnk5yoTQgkaWiqOlBMYg+NAJONAJH4ndjK46rfFUs9ieNp73X3X8fwPcASeCUMK', b'IhCUwX/qG5THqvZxGAZZwN+8vN+0jY4wju67Nvrw/u/IwsPUFYxSxSY/OCbZ9Q+W2Xwls3lSKISl', b'DWNcoQn0gMiEeC7AS4H4BBOhlOuFvKPrNhTU5QWDyc2xIlIp78gMproIWrKgJfdZDsjRkRw9jxyM', b'DM4F4pJ5yR2r7skF/HnJEYmZKxumw8CUqXwTkqEgXR8u3Iwg5fGzMZAMpr4IWrKgJfdZDkiykSQ7', b'lyTjlMAPKx/JY9U9yYA/P0lBhJtQASkZQGH8FiRDQQ4rmCOpIH23aiDpXYfugiVYBC1Z0JL7LAdE', b'+UiUn0cUSscJ40xJ4iN6rLonGvDnn1VTStRbEAwFZZVhnR7WLexf74OpLoKWLGjJfZYDcmIkJ84j', b'h7BknBBGEfWRO1bdkwv4+7XrYSgoux4K5iZ0GKXet6K7YKqLoCULWnKf5YBcOpJLzyFHsSKSEZoK', b'haSP3LHqnlzA3++1HoaChD6UyiEk2Gp4SQZTXwQtWdCS+ywHJOVIUp5HUvIUipLCv/fN5lh1TzLg', b'7/daD0NBut2Te7MhFMHbv49kMPVF0JIFLbnPMpBMXm157Zb+Q9E9lk0fPbTGtLXb0azb1mhQQe9g', b'Ut7oYjWeVHpt7GEKx92wkR5OTLt9+U6QjF8r5j8BUEsDBBQAAAAIAMSIYU+jsYa+zwEAAIkEAAAN', b'AAAAeGwvc3R5bGVzLnhtbJ1U226cQAz9FTQfUJaVGqkVIDWVIkVqq0ibh74OYGCkuZDBrCBfX89l', b'uWzzlBfGPtjn2B5DPuIi4dIDYDIrqceC9YjD9zQd6x4UH7+YATS9aY1VHMm1XToOFngzuiQl0/Pp', b'9JAqLjQrcz2pJ4VjUptJY8FOLEnLvDV6g84sABTLFSRXLgv2k0tRWRGCuRJyCfjZI7WRxiZI1UDB', b'Mg+N7yEgi64rNXIpoY31aBpkwrOKCfs3/hgpQkh5rI+AMh84Ilj9RE5I8uj/76L9ugxUX2f5kp2/', b'sl2GP0imMrYBexAKUJlLaNFlWNH13kAzuKMyiEY5qxG8M5qHSm5ph3R/kQXDni5i47pHHfE9FlXu', b'4Q8lo0Ht1CDlxaX8bdeeMuppbpMQ89z4BXDd30yadzTDokTHCezpAvmO9/w53kFcDT5O1J32/ttk', b'EF4stGL2/txuBRA9Hwa5/JCi0wpCNzvF7EPF7JOKZc5vOklvrHgnPrefNQFA63sFi6J2CF1YGNDc', b'ximtA/LjOsx+RZNqEhKFjmX1omkgVOS+uoL9cV+zPE5guwLiR17Rb+EgQFENtHyS+OK6vIR12exf', b'buGyhzXqdaUo2Gb/hkZM6ltQ3H4+5T9QSwMEFAAAAAgAxIhhTzzhpYXAAAAAFgIAAAsAAABfcmVs', b'cy8ucmVsc62ST4vCQAzFv0qZ+xpXwcNiPXnpbVn8AnEm/UM7kyETsX57h71st1RQ8Bhe8t6PR/Y/', b'NKB2HFLbxVSMfgipNK1q/AJItiWPacWRQlZqFo+aR2kgou2xIdis1zuQqYc57KeeReVKI5X7NMUJ', b'pSEtzTjAlaU/M/erbJuFW6RnQrmuO0tHthdPQReyZxumgGWYzR+MY/stnE8tC72G87gD8KToUPHX', b'9SPmABLtKD0k2i4QYYzv7odGpeDIzZHg3xsc7lBLAwQUAAAACADEiGFPsbVp6TIBAAApAgAADwAA', b'AHhsL3dvcmtib29rLnhtbI2Q3U7DMAyFX6XKA9BugklMKzdMwCQEiKHdp627WkviynE32NOTpCqM', b'O64c/+Q79lmdiA8V0SH7tMb5JZeqE+mXee7rDqz2V9SDC72W2GoJKe9zalusYU31YMFJPi+KRc5g', b'tCA532Hv1Uj7D8v3DLrxHYBYM6KsRqfuVtNmb5zllxkJ1FEpVmNlh3DyvwMxzXSYOMKHrkpVqEwP', b'Qg9oBHitBR6Zhh7dvlQzlbXIXrZRPU1adGjxDE3KfEenJ2I8kxNttjWTMelXbKRPQcH/VHbAgvWf', b'QdHVe7SlVIsiAI/osUKD8lWq9Dag4hn5xR3JiilmTlsoVdJKKiFumlFRwi0XGF5iaPCmmY3IidNA', b'iw6alwDysREWrIOlMSTS/PpmdhuMGIy5D7VX90x6VIiUydW7b1BLAwQUAAAACADEiGFPoS02da8A', b'AAD7AQAAGgAAAHhsL19yZWxzL3dvcmtib29rLnhtbC5yZWxzrZHPCoMwDIdfpfQBFnWww1BPu3gd', b'e4Gi8Q+2tjQZ07df6Q46cLCDl4aQ8Ps+0vyOWvFgJ+oHR2I2eqJC9szuCkB1j0bRyTqcwqS13igO', b're/AqXpUHUKWJBfw2wxZ5ttMUTWF9FWTSvFQvkMuJMwaXtaP1CMyQSzpKQDCyuLwH7xt26HGm62f', b'BifesVgBUsC+UbYaES8a6WiFT+pP/nnlc6AgxPfwQ8TUKAFff12+AVBLAwQUAAAACADEiGFP9ou5', b'IBUBAADXAwAAEwAAAFtDb250ZW50X1R5cGVzXS54bWytU8tOwzAQ/JUoV5S4cOCAml4oV+iBHzD2', b'prHql7zbkv49G6dUogqoqLnYXs/uzNhrL9+PEbDonfXYlB1RfBICVQdOYh0ieEbakJwkDtNWRKl2', b'cgviYbF4FCp4Ak8VDRzlarmGVu4tFc/j/kDdlDJGa5QkE7w4eH1BWp0I6wQ252BnIt5xQlm89MyC', b'vNeUjGJZiCskLitzzIVvB0jJaPiXudC2RoEOau+4pMaYQGrsAMjZGulo4WR1IxO9SseMordiROo5', b'pYk7AlNaGRjH+9sUvzuhQoIqJkYTmYkDsqcNoyiGxFkPCUPbNOir1Jl6VvGfzf0MaZfXU3d+BlHk', b'6caL//uVDXPtpPG/OfkIYXc2IPJ/Xn0BUEsBAhQAFAAAAAgAxIhhTwdBTWKBAAAAsQAAABAAAAAA', b'AAAAAAAAAIABAAAAAGRvY1Byb3BzL2FwcC54bWxQSwECFAAUAAAACADEiGFPzv2fo+oAAADLAQAA', b'EQAAAAAAAAAAAAAAgAGvAAAAZG9jUHJvcHMvY29yZS54bWxQSwECFAAUAAAACADEiGFPmVycIxAG', b'AACcJwAAEwAAAAAAAAAAAAAAgAHIAQAAeGwvdGhlbWUvdGhlbWUxLnhtbFBLAQIUABQAAAAIAMSI', b'YU/mba7xrAMAALkQAAAYAAAAAAAAAAAAAAC2gQkIAAB4bC93b3Jrc2hlZXRzL3NoZWV0MS54bWxQ', b'SwECFAAUAAAACADEiGFPo7GGvs8BAACJBAAADQAAAAAAAAAAAAAAgAHrCwAAeGwvc3R5bGVzLnht', b'bFBLAQIUABQAAAAIAMSIYU884aWFwAAAABYCAAALAAAAAAAAAAAAAACAAeUNAABfcmVscy8ucmVs', b'c1BLAQIUABQAAAAIAMSIYU+xtWnpMgEAACkCAAAPAAAAAAAAAAAAAACAAc4OAAB4bC93b3JrYm9v', b'ay54bWxQSwECFAAUAAAACADEiGFPoS02da8AAAD7AQAAGgAAAAAAAAAAAAAAgAEtEAAAeGwvX3Jl', b'bHMvd29ya2Jvb2sueG1sLnJlbHNQSwECFAAUAAAACADEiGFP9ou5IBUBAADXAwAAEwAAAAAAAAAA', b'AAAAgAEUEQAAW0NvbnRlbnRfVHlwZXNdLnhtbFBLBQYAAAAACQAJAD4CAABaEgAAAAA=', b'', b'--===============2997002501849855988==--']

In [68]:
x = b"aml1Z3VhaeS9oOWlvQ==?="
import base64
base64.b64decode(x).decode()

'jiuguai你好'

In [48]:
from pprint import pprint
pprint(l)


[b'Date: Sat, 2 Nov 2019 16:41:18 +0800',
 b'X-QQ-mid: esmtp7t1572684076tzpuyvle2',
 b'Received: from [192.168.0.100] (unknown [175.10.95.63])',
 b'\tby esmtp4.qq.com (ESMTP) with SMTP id 0',
 b'\tfor <2437253053@qq.com>; Sat, 02 Nov 2019 16:41:16 +0800 (CST)',
 b'X-QQ-SSF: A001000000000010G3120000002000J',
 b'X-QQ-FEAT: DvNdkFXOZE+xrKCpNhH6civIzs0DRP9Dq8D6fRROQnsZyOs/UPqYKlP9LuFy6',
 b'\tba5T4CpBzLi7JQwXnL67wV+jsoRmkhUptPoJQLtxS1Z9yz2EeVLkcEpg+4di4EnKIG0zieH',
 b'\tZrIXwXw1oFAM7z8fuHni71R9S5EdvsHpt1xUhQh/NciLFPPjYPSZVGpMKesBzHIBcb2cyRg',
 b'\tuIRD01Fr30Bl3rX2+u4CuJ0FTYeweZCPwhSm8ntamawRWFI2L2c8LA37JJ2j0pU8=',
 b'X-QQ-CSender: 2437253053@qq.com',
 b'Sender: 2437253053@qq.com',
 b'Content-Type: multipart/mixed; boundary="===============2997002501849855988='
 b'="',
 b'MIME-Version: 1.0',
 b'From: =?utf-8?b?WkVST+aVmeeoiw==?=',
 b'To: =?utf-8?b?aml1Z3VhaeS9oOWlvQ==?=',
 b'Subject: =?utf-8?b?UHl0aG9uIFNNVFAg6YKu5Lu25rWL6K+V?=',
 b'',
 b'--===============2997002501849855988==',
 b'Content-

In [43]:
resp

b'+OK'

In [41]:
mails

[b'1 8305', b'2 8308', b'3 8518']

In [33]:
print(len(lines))

119


In [66]:
import base64

base64.b64decode(b"=anVzdHpjYw==?=").decode("gb18030")

'justzcc'

In [ ]:
stop


In [ ]:
# 方法一

import poplib
 
# base64解码
import base64
 
# 解码邮件信息
from email.parser import Parser
 
# 解析邮件主题
from email.header import decode_header
 
# 解析发件人详情，名称及地址
import base64


from email.utils import parseaddr


In [ ]:
usercconunt = "wind2zero@qq.com"
password = "vclmmuvcnwntdjef"

pop3_server = 'pop.qq.com'



server = poplib.POP3(pop3_server)



server.set_debuglevel(1)
print(server.getwelcome().decode('utf8'))
server.user(usercconunt)
server.pass_(password)



email_num, email_size = server.stat()


rsp, msg_list, rsp_siz = server.list()
print("服务器的响应: {0},\n消息列表： {1},\n返回消息的大小： {2}".format(rsp, msg_list, rsp_siz))
 
print('邮件总数： {}'.format(len(msg_list)))

total_mail_numbers = len(msg_list)
rsp, msglines, msgsiz = server.retr(total_mail_numbers)
#print("服务器的响应: {0},\n原始邮件内容： {1},\n该封邮件所占字节大小： {2}".format(rsp, msglines, msgsiz))

msg_content = b'\r\n'.join(msglines).decode('gbk')

msg = Parser().parsestr(text=msg_content)
print('解码后的邮件信息:\n{}'.format(msg))



In [ ]:
content = msg.get_payload()

In [ ]:
content[1].as_string().split('base64')

In [ ]:
content_charset = content[0].get_content_charset() # 获取编码格式
text = content[0].as_string().split('base64')[-1]


text_content = base64.b64decode(text).decode(content_charset) # base64解码


In [ ]:
content_charset = content[1].get_content_charset()
content[1].as_string().split('base64')[-1]



In [ ]:
content[1].as_string()

In [ ]:
def parser_content(msg):
    content = msg.get_payload()

    # 文本信息
    content_charset = content[0].get_content_charset() # 获取编码格式
    text = content[0].as_string().split('base64')[-1]
    text_content = base64.b64decode(text).decode(content_charset) # base64解码

    # 添加了HTML代码的信息
    content_charset = content[1].get_content_charset()
    text = content[1].as_string().split('base64')[-1]
    html_content = base64.b64decode(text).decode(content_charset)

    print('文本信息: {0}\n添加了HTML代码的信息: {1}'.format(text_content, html_content))


In [ ]:
parser_content(msg)

In [ ]:
file_path = r"D:\Downloads\Apple.xlsx"
pd.read_excel(file_path)